In [177]:
from fastai.vision import *

In [178]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

### No Acc

In [179]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.258781,0.170574,0.939156,00:03


In [180]:
data.batch_size

64

### Naive Acc 

In [181]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,1.080031,7.560116,0.497056,00:51


### Acc + BnFreeze

In [182]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32), BnFreeze])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,1.068713,4.552332,0.589303,00:51


### Increase Momentum 

In [183]:
def set_bn_mom(m:nn.Module, mom=0.9):
    "Set bn layers in eval mode for all recursive children of `m`."
    for l in m.children():
        if isinstance(l, bn_types):
            l.momentum = mom
        set_bn_mom(l, mom)

In [184]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.9)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,1.200330,786.593262,0.494603,00:51


### Decrease Momentum

In [185]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.01)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,1.134230,3.278407,0.563788,00:51


### InstanceNorm

In [186]:
def bn2instance(bn):
    if isinstance(bn, nn.BatchNorm1d):
        inst = nn.InstanceNorm1d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm2d):
        inst = nn.InstanceNorm2d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm3d):
        inst = nn.InstanceNorm3d(bn.num_features, affine=True)
    
    inst.weight = bn.weight
    inst.bias = bn.bias
    inst.running_mean = nn.Parameter(bn.running_mean, requires_grad=False)
    inst.running_var = nn.Parameter(bn.running_var, requires_grad=False)
    inst.momentum = bn.momentum
    inst.eps = bn.eps
    return (inst).to(bn.weight.device)

In [187]:
def convert_bn(list_mods, func=bn2instance):
    for i in range(len(list_mods)):
        if isinstance(list_mods[i], bn_types):
            list_mods[i] = func(list_mods[i])
        elif list_mods[i].__class__.__name__ in ("Sequential", "BasicBlock"):
            list_mods[i] = nn.Sequential(*convert_bn(list(list_mods[i].children()), func))
    return list_mods

In [188]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [189]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2instance))

In [190]:
learn.freeze()

In [191]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [192]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,1.383479,1.387812,0.504416,01:20


### GroupNorm

In [193]:
def bn2group(bn):
    groupnorm = nn.GroupNorm(4, bn.num_features, affine=True)
    groupnorm.weight = bn.weight
    groupnorm.bias = bn.bias
    groupnorm.running_mean = nn.Parameter(bn.running_mean, requires_grad=False)
    groupnorm.running_var = nn.Parameter(bn.running_var, requires_grad=False)
    groupnorm.momentum = bn.momentum
    groupnorm.eps = bn.eps
    return (groupnorm).to(bn.weight.device)

In [194]:
data = ImageDataBunch.from_folder(path, bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [195]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2group))

In [196]:
learn.freeze()

In [197]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.163103,0.140408,0.977429,01:00
